# Lensing Systems Positions Table

In [1]:
# make a table in LaTeX format for easy pasting

import h5py
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Plots.model_plot import ModelPlot

# define filters and system names
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

center_ra = 3.99999999999998
center_dec = 1.9984014443252818e-15

# table format:
# Lens System Name, RA_lens, Dec_lens, RA_im1, Dec_im1, Ra_im2, Dec_im2, Image Separation [arcsec]
for i, name in enumerate(names):
    filename = f"../../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"

    with h5py.File(filename, "r") as f:
        kwargs = {}
        for key in f:
            kwargs[key] = f[key][()]

    ra_lens = kwargs['center_x'] - center_ra
    dec_lens = kwargs['center_y'] - center_dec

    ra_im1 = kwargs['image_position'][0][0] - center_ra
    dec_im1 = kwargs['image_position'][1][0] - center_dec

    ra_im2 = kwargs['image_position'][0][1] - center_ra
    dec_im2 = kwargs['image_position'][1][1] - center_dec

    image_sep = np.sqrt((kwargs['image_position'][0][0] - kwargs['image_position'][0][1])**2 + (kwargs['image_position'][1][0] - kwargs['image_position'][1][1])**2)

    # print the data in LaTeX format
    print(f"{name} & {ra_lens:.3f} & {dec_lens:.3f} & {ra_im1:.3f} & {dec_im1:.3f} & {ra_im2:.3f} & {dec_im2:.3f} & {image_sep:.3f} \\\\")

J0407-5006 & -0.544 & -0.680 & -0.709 & -0.931 & 0.189 & 0.539 & 1.722 \\
J0806+2006 & 0.191 & -0.157 & -0.738 & 0.344 & 0.581 & -0.342 & 1.487 \\
J1442+4055 & 0.234 & -0.265 & -1.066 & 0.129 & 0.992 & -0.428 & 2.132 \\
J1515+1511 & 0.553 & -0.211 & -0.847 & 0.638 & 0.821 & -0.480 & 2.008 \\
J1620+1203 & -0.726 & -0.252 & -1.065 & -0.652 & 1.042 & 1.208 & 2.810 \\
J2325-5229 & -0.206 & -0.364 & -0.693 & 1.294 & 0.706 & -1.156 & 2.821 \\


# Magnitude Table

In [17]:
import h5py
import numpy as np
import os

# Define filters and system names
names_f814w = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

# LaTeX table header
print("\\begin{tabular}{lccc|ccc}")
print("\\hline")
print("\\multicolumn{4}{c|}{F814W Magnitude} & \\multicolumn{3}{c}{F475X Magnitude} \\\\")
print("\\hline")
print("Lens System & Lens & Image A & Image B & Lens & Image A & Image B \\\\")
print("\\hline")

# Table content
for name in names_f814w:
    # F814W file
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W_final.hdf5"
    with h5py.File(filename_f814w, "r") as f:
        kwargs_f814w = {key: f[key][()] for key in f}

    # Try to open F475X file if it exists
    filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X_final.hdf5"
    has_f475x = os.path.exists(filename_f475x)
    if has_f475x:
        with h5py.File(filename_f475x, "r") as f:
            kwargs_f475x = {key: f[key][()] for key in f}

    # --- F814W ---
    lens_flux = kwargs_f814w['lens_flux']
    lens_flux_sigma = kwargs_f814w['lens_flux_sigma']
    lens_mag = kwargs_f814w['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    im1_flux = kwargs_f814w['image_fluxes'][0]
    im1_flux_sigma = kwargs_f814w['image_fluxes_sigma'][0]
    im1_mag = kwargs_f814w['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    im2_flux = kwargs_f814w['image_fluxes'][1]
    im2_flux_sigma = kwargs_f814w['image_fluxes_sigma'][1]
    im2_mag = kwargs_f814w['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    # --- F475X (if available) ---
    if has_f475x:
        lens_flux_f475x = kwargs_f475x['lens_flux']
        lens_flux_sigma_f475x = kwargs_f475x['lens_flux_sigma']
        lens_mag_f475x = kwargs_f475x['lens_mag']
        lens_mag_sigmas_f475x = mag_sigma(lens_flux_f475x, lens_flux_sigma_f475x)

        im1_flux_f475x = kwargs_f475x['image_fluxes'][0]
        im1_flux_sigma_f475x = kwargs_f475x['image_fluxes_sigma'][0]
        im1_mag_f475x = kwargs_f475x['image_mags'][0]
        im1_mag_sigmas_f475x = mag_sigma(im1_flux_f475x, im1_flux_sigma_f475x)

        im2_flux_f475x = kwargs_f475x['image_fluxes'][1]
        im2_flux_sigma_f475x = kwargs_f475x['image_fluxes_sigma'][1]
        im2_mag_f475x = kwargs_f475x['image_mags'][1]
        im2_mag_sigmas_f475x = mag_sigma(im2_flux_f475x, im2_flux_sigma_f475x)

        f475x_lens = f"${lens_mag_f475x:.3f}^{{+{lens_mag_sigmas_f475x[0]:.3f}}}_{{-{lens_mag_sigmas_f475x[1]:.3f}}}$"
        f475x_im1 = f"${im1_mag_f475x:.4f}^{{+{im1_mag_sigmas_f475x[0]:.4f}}}_{{-{im1_mag_sigmas_f475x[1]:.4f}}}$"
        f475x_im2 = f"${im2_mag_f475x:.4f}^{{+{im2_mag_sigmas_f475x[0]:.4f}}}_{{-{im2_mag_sigmas_f475x[1]:.4f}}}$"
    else:
        f475x_lens = f475x_im1 = f475x_im2 = "--"

    # --- Print table row ---
    print(f"{name} & "
          f"${lens_mag:.3f}^{{+{lens_mag_sigmas[0]:.3f}}}_{{-{lens_mag_sigmas[1]:.3f}}}$ & "
          f"${im1_mag:.4f}^{{+{im1_mag_sigmas[0]:.4f}}}_{{-{im1_mag_sigmas[1]:.4f}}}$ & "
          f"${im2_mag:.4f}^{{+{im2_mag_sigmas[0]:.4f}}}_{{-{im2_mag_sigmas[1]:.4f}}}$ & "
          f"{f475x_lens} & {f475x_im1} & {f475x_im2} \\\\")

print("\\hline")
print("\\end{tabular}")


\begin{tabular}{lccc|ccc}
\hline
\multicolumn{4}{c|}{F814W Magnitude} & \multicolumn{3}{c}{F475X Magnitude} \\
\hline
Lens System & Lens & Image A & Image B & Lens & Image A & Image B \\
\hline
J0407-5006 & $20.300^{+0.006}_{-0.007}$ & $19.4233^{+0.0034}_{-0.0025}$ & $18.0127^{+0.0004}_{-0.0005}$ & -- & -- & -- \\
J0806+2006 & $19.900^{+0.007}_{-0.007}$ & $19.2485^{+0.0002}_{-0.0002}$ & $20.0683^{+0.0006}_{-0.0006}$ & $22.659^{+0.054}_{-0.055}$ & $20.0233^{+0.0004}_{-0.0004}$ & $20.7323^{+0.0008}_{-0.0008}$ \\
J1442+4055 & $19.056^{+0.002}_{-0.003}$ & $18.1598^{+0.0001}_{-0.0001}$ & $18.9897^{+0.0002}_{-0.0002}$ & $20.818^{+0.005}_{-0.005}$ & $18.8834^{+0.0001}_{-0.0002}$ & $19.8187^{+0.0005}_{-0.0005}$ \\
J1515+1511 & $21.436^{+0.014}_{-0.014}$ & $18.1740^{+0.0000}_{-0.0001}$ & $18.5752^{+0.0002}_{-0.0002}$ & -- & -- & -- \\
J1620+1203 & $18.946^{+0.002}_{-0.002}$ & $20.5387^{+0.0006}_{-0.0007}$ & $19.0400^{+0.0002}_{-0.0001}$ & $21.162^{+0.006}_{-0.006}$ & $20.8910^{+0.0004}_{-0.0004

# Lens Light Parameter Table

In [18]:
import pickle
import numpy as np
import h5py

names_f814w = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']

# LaTeX header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{F814W and F475X Best-fit Lens Light Model Parameters}")
print("\\label{tab:lens_light_params}")
print("\\begin{tabular}{lccc|ccc}")
print("\\hline")
print("\\multicolumn{4}{c|}{\\textbf{F814W}} & \\multicolumn{3}{c}{\\textbf{F475X}} \\\\")
print("\\hline")
print("Lens System & $R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q$ & $R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q$ \\\\")
print("\\hline")

for name in names_f814w:
    # --- F814W Data ---
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W_fit.pkl"
    with open(filename_f814w, "rb") as f:
        loaded_data_f814w = pickle.load(f)

    chain_list_f814w = loaded_data_f814w.get('chain_list')
    sampler_type, samples_mcmc_f814w, param_mcmc_f814w, dist_mcmc_f814w = chain_list_f814w[1]

    param_names = ['R_sersic_lens_light0', 'n_sersic_lens_light0', 'e1_lens_light0', 'e2_lens_light0']
    indices_f814w = [param_mcmc_f814w.index(p) for p in param_names]

    r_sersic_chain_f814w = samples_mcmc_f814w[:, indices_f814w[0]]
    n_sersic_chain_f814w = samples_mcmc_f814w[:, indices_f814w[1]]
    e1_chain_f814w = samples_mcmc_f814w[:, indices_f814w[2]]
    e2_chain_f814w = samples_mcmc_f814w[:, indices_f814w[3]]

    e_chain_f814w = np.sqrt(e1_chain_f814w**2 + e2_chain_f814w**2)
    q_chain_f814w = (1 - e_chain_f814w) / (1 + e_chain_f814w)

    def get_median_and_uncertainties(samples):
        median = np.percentile(samples, 50)
        lower = median - np.percentile(samples, 16)
        upper = np.percentile(samples, 84) - median
        return median, lower, upper

    r_median_f814w, r_lower_f814w, r_upper_f814w = get_median_and_uncertainties(r_sersic_chain_f814w)
    n_median_f814w, n_lower_f814w, n_upper_f814w = get_median_and_uncertainties(n_sersic_chain_f814w)
    q_median_f814w, q_lower_f814w, q_upper_f814w = get_median_and_uncertainties(q_chain_f814w)

    # --- F475X Data (only if available) ---
    if name in names_f475x:
        filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X_fit.pkl"
        with open(filename_f475x, "rb") as f:
            loaded_data_f475x = pickle.load(f)

        chain_list_f475x = loaded_data_f475x.get('chain_list')
        sampler_type, samples_mcmc_f475x, param_mcmc_f475x, dist_mcmc_f475x = chain_list_f475x[1]

        indices_f475x = [param_mcmc_f475x.index(p) for p in param_names]

        r_sersic_chain_f475x = samples_mcmc_f475x[:, indices_f475x[0]]
        n_sersic_chain_f475x = samples_mcmc_f475x[:, indices_f475x[1]]
        e1_chain_f475x = samples_mcmc_f475x[:, indices_f475x[2]]
        e2_chain_f475x = samples_mcmc_f475x[:, indices_f475x[3]]

        e_chain_f475x = np.sqrt(e1_chain_f475x**2 + e2_chain_f475x**2)
        q_chain_f475x = (1 - e_chain_f475x) / (1 + e_chain_f475x)

        r_median_f475x, r_lower_f475x, r_upper_f475x = get_median_and_uncertainties(r_sersic_chain_f475x)
        n_median_f475x, n_lower_f475x, n_upper_f475x = get_median_and_uncertainties(n_sersic_chain_f475x)
        q_median_f475x, q_lower_f475x, q_upper_f475x = get_median_and_uncertainties(q_chain_f475x)

        r_str_f475x = f"${r_median_f475x:.2f}^{{+{r_upper_f475x:.2f}}}_{{-{r_lower_f475x:.2f}}}$"
        n_str_f475x = f"${n_median_f475x:.2f}^{{+{n_upper_f475x:.2f}}}_{{-{n_lower_f475x:.2f}}}$"
        q_str_f475x = f"${q_median_f475x:.3f}^{{+{q_upper_f475x:.3f}}}_{{-{q_lower_f475x:.3f}}}$"
    else:
        r_str_f475x = "$--$"
        n_str_f475x = "$--$"
        q_str_f475x = "$--$"

    # Print table row
    print(f"{name} & "
          f"${r_median_f814w:.2f}^{{+{r_upper_f814w:.2f}}}_{{-{r_lower_f814w:.2f}}}$ & "
          f"${n_median_f814w:.2f}^{{+{n_upper_f814w:.2f}}}_{{-{n_lower_f814w:.2f}}}$ & "
          f"${q_median_f814w:.3f}^{{+{q_upper_f814w:.3f}}}_{{-{q_lower_f814w:.3f}}}$ & "
          f"{r_str_f475x} & {n_str_f475x} & {q_str_f475x} \\\\")

print("\\hline")
print("\\end{tabular}")
print("\\end{table*}")


\begin{table*}[htb]
\centering
\caption{F814W and F475X Best-fit Lens Light Model Parameters}
\label{tab:lens_light_params}
\begin{tabular}{lccc|ccc}
\hline
\multicolumn{4}{c|}{\textbf{F814W}} & \multicolumn{3}{c}{\textbf{F475X}} \\
\hline
Lens System & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ \\
\hline
J0407-5006 & $0.47^{+0.01}_{-0.01}$ & $4.96^{+0.00}_{-0.00}$ & $0.728^{+0.002}_{-0.002}$ & $--$ & $--$ & $--$ \\
J0806+2006 & $0.50^{+0.01}_{-0.01}$ & $3.87^{+0.02}_{-0.02}$ & $0.926^{+0.006}_{-0.006}$ & $0.30^{+0.02}_{-0.02}$ & $2.19^{+0.04}_{-0.04}$ & $0.695^{+0.009}_{-0.009}$ \\
J1442+4055 & $0.67^{+0.01}_{-0.01}$ & $4.96^{+0.01}_{-0.01}$ & $0.830^{+0.004}_{-0.004}$ & $0.90^{+0.01}_{-0.01}$ & $4.95^{+0.01}_{-0.01}$ & $0.907^{+0.006}_{-0.005}$ \\
J1515+1511 & $0.52^{+0.02}_{-0.02}$ & $1.79^{+0.02}_{-0.02}$ & $0.158^{+0.003}_{-0.003}$ & $--$ & $--$ & $--$ \\
J1620+1203 & $1.72^{+0.02}_{-0.02}$ & $4.97^{+

In [19]:
import pickle
import numpy as np
import os

names_f814w = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
names_f475x = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']

# LaTeX table header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{F814W and F475X best-fit lens mass parameters.}")
print("\\label{tab:lens_mass_params}")
print("\\begin{tabular}{lccc|ccc}")
print("\\hline")
print("\\multicolumn{4}{c|}{{F814W}} & \\multicolumn{3}{c}{{F475X}} \\\\")
print("\\hline")
print("Lens System & $\\theta_E$ [arcsec] & $q_{\\text{mass}}$ & $\\gamma$ & $\\theta_E$ & $q_{\\text{mass}}$ & $\\gamma$ \\\\")
print("\\hline")

def get_median_and_uncertainties(samples):
    median = np.percentile(samples, 50)
    lower = median - np.percentile(samples, 16)
    upper = np.percentile(samples, 84) - median
    return median, lower, upper

for name in names_f814w:
    filename_f814w = f"../../cutout_data/{name}/F814W/{name}_F814W_fit.pkl"
    filename_f475x = f"../../cutout_data/{name}/F475X/{name}_F475X_fit.pkl"

    # F814W
    with open(filename_f814w, "rb") as f:
        loaded_data_f814w = pickle.load(f)
    chain_list_f814w = loaded_data_f814w.get('chain_list')
    sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list_f814w[1]

    param_names_mass = ['theta_E_lens0', 'e1_lens0', 'e2_lens0', 'gamma1_lens1', 'gamma2_lens1']
    indices_f814w = [param_mcmc.index(p) for p in param_names_mass]

    theta_E_chain = samples_mcmc[:, indices_f814w[0]]
    e1_chain = samples_mcmc[:, indices_f814w[1]]
    e2_chain = samples_mcmc[:, indices_f814w[2]]
    gamma1_chain = samples_mcmc[:, indices_f814w[3]]
    gamma2_chain = samples_mcmc[:, indices_f814w[4]]

    # Mass axis ratio from ellipticity
    e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
    q_mass_chain = (1 - e_chain) / (1 + e_chain)

    # External shear strength
    gamma_chain = np.sqrt(gamma1_chain**2 + gamma2_chain**2)

    theta_E_median, theta_E_lower, theta_E_upper = get_median_and_uncertainties(theta_E_chain)
    q_mass_median, q_mass_lower, q_mass_upper = get_median_and_uncertainties(q_mass_chain)
    gamma_median, gamma_lower, gamma_upper = get_median_and_uncertainties(gamma_chain)

    # Initialize empty values for F475X
    theta_E_f475x = q_mass_f475x = gamma_f475x = '...'

    if name in names_f475x and os.path.exists(filename_f475x):
        with open(filename_f475x, "rb") as f:
            loaded_data_f475x = pickle.load(f)
        chain_list_f475x = loaded_data_f475x.get('chain_list')
        sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list_f475x[1]

        indices_f475x = [param_mcmc.index(p) for p in param_names_mass]

        theta_E_chain_f475x = samples_mcmc[:, indices_f475x[0]]
        e1_chain_f475x = samples_mcmc[:, indices_f475x[1]]
        e2_chain_f475x = samples_mcmc[:, indices_f475x[2]]
        gamma1_chain_f475x = samples_mcmc[:, indices_f475x[3]]
        gamma2_chain_f475x = samples_mcmc[:, indices_f475x[4]]

        e_chain_f475x = np.sqrt(e1_chain_f475x**2 + e2_chain_f475x**2)
        q_mass_chain_f475x = (1 - e_chain_f475x) / (1 + e_chain_f475x)

        gamma_chain_f475x = np.sqrt(gamma1_chain_f475x**2 + gamma2_chain_f475x**2)

        theta_E_median_f475x, theta_E_lower_f475x, theta_E_upper_f475x = get_median_and_uncertainties(theta_E_chain_f475x)
        q_mass_median_f475x, q_mass_lower_f475x, q_mass_upper_f475x = get_median_and_uncertainties(q_mass_chain_f475x)
        gamma_median_f475x, gamma_lower_f475x, gamma_upper_f475x = get_median_and_uncertainties(gamma_chain_f475x)

        theta_E_f475x = f"${theta_E_median_f475x:.3f}^{{+{theta_E_upper_f475x:.3f}}}_{{-{theta_E_lower_f475x:.3f}}}$"
        q_mass_f475x = f"${q_mass_median_f475x:.3f}^{{+{q_mass_upper_f475x:.3f}}}_{{-{q_mass_lower_f475x:.3f}}}$"
        gamma_f475x = f"${gamma_median_f475x:.3f}^{{+{gamma_upper_f475x:.3f}}}_{{-{gamma_lower_f475x:.3f}}}$"

    print(f"{name} & "
          f"${theta_E_median:.3f}^{{+{theta_E_upper:.3f}}}_{{-{theta_E_lower:.3f}}}$ & "
          f"${q_mass_median:.3f}^{{+{q_mass_upper:.3f}}}_{{-{q_mass_lower:.3f}}}$ & "
          f"${gamma_median:.3f}^{{+{gamma_upper:.3f}}}_{{-{gamma_lower:.3f}}}$ & "
          f"{theta_E_f475x} & "
          f"{q_mass_f475x} & "
          f"{gamma_f475x} \\\\")

print("\\hline")
print("\\end{tabular}")
print("\\end{table*}")


\begin{table*}[htb]
\centering
\caption{F814W and F475X best-fit lens mass parameters.}
\label{tab:lens_mass_params}
\begin{tabular}{lccc|ccc}
\hline
\multicolumn{4}{c|}{{F814W}} & \multicolumn{3}{c}{{F475X}} \\
\hline
Lens System & $\theta_E$ [arcsec] & $q_{\text{mass}}$ & $\gamma$ & $\theta_E$ & $q_{\text{mass}}$ & $\gamma$ \\
\hline
J0407-5006 & $0.672^{+0.002}_{-0.004}$ & $0.687^{+0.016}_{-0.009}$ & $0.280^{+0.003}_{-0.002}$ & ... & ... & ... \\
J0806+2006 & $0.781^{+0.004}_{-0.004}$ & $0.842^{+0.013}_{-0.015}$ & $0.089^{+0.004}_{-0.004}$ & $0.763^{+0.008}_{-0.008}$ & $0.748^{+0.018}_{-0.016}$ & $0.103^{+0.009}_{-0.011}$ \\
J1442+4055 & $0.959^{+0.003}_{-0.003}$ & $0.784^{+0.012}_{-0.015}$ & $0.097^{+0.003}_{-0.003}$ & $1.081^{+0.006}_{-0.005}$ & $0.776^{+0.016}_{-0.014}$ & $0.056^{+0.006}_{-0.004}$ \\
J1515+1511 & $0.998^{+0.008}_{-0.008}$ & $0.589^{+0.014}_{-0.016}$ & $0.155^{+0.006}_{-0.006}$ & ... & ... & ... \\
J1620+1203 & $1.558^{+0.010}_{-0.008}$ & $0.771^{+0.016}_{-0.018}$

In [2]:
import h5py
import numpy as np

# define filters and system names
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

print('FB / FA:')

# Table content
for name in names:
    filename = f"../../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"
    with h5py.File(filename, "r") as f:
        kwargs = {key: f[key][()] for key in f}

    # Lens
    lens_flux = kwargs['lens_flux']
    lens_flux_sigma = kwargs['lens_flux_sigma']
    lens_mag = kwargs['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    # Image A
    im1_flux = kwargs['image_fluxes'][0]
    im1_flux_sigma = kwargs['image_fluxes_sigma'][0]
    im1_mag = kwargs['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    # Image B
    im2_flux = kwargs['image_fluxes'][1]
    im2_flux_sigma = kwargs['image_fluxes_sigma'][1]
    im2_mag = kwargs['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    print(f'{name}: {im2_flux / im1_flux:.3f}')

FB / FA:
J0407-5006: 3.671
J0806+2006: 0.470
J1442+4055: 0.466
J1515+1511: 0.691
J1620+1203: 3.977
J2325-5229: 0.689


J0806+2006: 0.520
J1442+4055: 0.422
J1620+1203: 3.841
J2325-5229: 0.560

In [4]:
import h5py
import numpy as np

# define filters and system names
names = ['J0806+2006', 'J1442+4055', 'J1620+1203', 'J2325-5229']
filter = 'F475X'

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

print('FB / FA:')

# Table content
for name in names:
    filename = f"../../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"
    with h5py.File(filename, "r") as f:
        kwargs = {key: f[key][()] for key in f}

    # Lens
    lens_flux = kwargs['lens_flux']
    lens_flux_sigma = kwargs['lens_flux_sigma']
    lens_mag = kwargs['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    # Image A
    im1_flux = kwargs['image_fluxes'][0]
    im1_flux_sigma = kwargs['image_fluxes_sigma'][0]
    im1_mag = kwargs['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    # Image B
    im2_flux = kwargs['image_fluxes'][1]
    im2_flux_sigma = kwargs['image_fluxes_sigma'][1]
    im2_mag = kwargs['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    print(f'{name}: {im2_flux / im1_flux:.3f}')

FB / FA:
J0806+2006: 0.520
J1442+4055: 0.422
J1620+1203: 3.841
J2325-5229: 0.560
